# Análisis de datos de dataset SNBE al 2023_04

Hoy (2023-04-06) me enteré, [por un reel](https://www.instagram.com/reel/Cqn3HJ-O0wv/), que se consiguieron los datos del **Sistema Nacional de Billetaje Electrónico** (SNBE) asi que me puse en contacto con Belén y Marcelo para poder hecharles una mirada.

En el reel dice que son unos 8 CDs 🤪, y cuando obtuve acceso a los datos me encontré con un archio `.rar` partido en 8 (jeje). Al descomprimirlo aparecieron 15 archivos `.csv`.

## Cuantos archivos tenemos?

In [1]:
%%bash
find data/source/INFORMACION\ SNBE/ -type f  -exec sha256sum {} \;

2b6aba2207ce5595db34050612aeb0cc976a9f3a4d555ce4d7cfb5ec9c0952b3  data/source/INFORMACION SNBE/1- Enero_2022.csv
e31c9de1930c0fdd230889a7737473413e19f3c16e547134e81afd6b8b5d1baf  data/source/INFORMACION SNBE/10- Octubre_2022.csv
d9314f96967e63b16e4f9afb68cb637727d0177dccbb609086b9404537a41702  data/source/INFORMACION SNBE/11- Noviembre_2022.csv
8bf8d04e534c33be1d0520d919cdecbaa9d6ecda152b0cb271dcbd7395195457  data/source/INFORMACION SNBE/12- Diciembre_2022.csv
855e03d373d76466b658511a11b07f5727ee5cfa821ca049de974e22800d430b  data/source/INFORMACION SNBE/13- Enero_2023.csv
1784c6ddc5b3efab34903c5ba4fb274de96bff789d13d3c9f6875906029e57d1  data/source/INFORMACION SNBE/14- Febrero_2023.csv
f54b8e6fbe7fcdc00454c072f6492ac3da3e85057e1d8f8bb56312e0aa61a740  data/source/INFORMACION SNBE/15- Marzo_2023.csv
5c99d08625919ae26d314b552e04e2b013278468ae24a7cbc27d832f40127cab  data/source/INFORMACION SNBE/2- Febrero_2022.csv
deffda61b82438a569afb0182232b476ab080757cd550cfe40cef886e120f03c  data/sourc

Un archivo por mes desde Enero del 2022 hasta Marzo del 2023. Supongo que está bien teniendo en cuenta que tienen que ser muchísimos registros, y como son `csv` son texto plano.

## Cuanto será que pesan los archivos?

In [2]:
%%bash
du -bsh data/source/INFORMACION\ SNBE/*

1.8G	data/source/INFORMACION SNBE/10- Octubre_2022.csv
1.8G	data/source/INFORMACION SNBE/11- Noviembre_2022.csv
1.8G	data/source/INFORMACION SNBE/12- Diciembre_2022.csv
1.6G	data/source/INFORMACION SNBE/13- Enero_2023.csv
1.5G	data/source/INFORMACION SNBE/14- Febrero_2023.csv
1.9G	data/source/INFORMACION SNBE/15- Marzo_2023.csv
1.3G	data/source/INFORMACION SNBE/1- Enero_2022.csv
1.4G	data/source/INFORMACION SNBE/2- Febrero_2022.csv
1.7G	data/source/INFORMACION SNBE/3- Marzo_2022.csv
1.6G	data/source/INFORMACION SNBE/4- Abril_2022.csv
1.7G	data/source/INFORMACION SNBE/5- Mayo_2022.csv
1.7G	data/source/INFORMACION SNBE/6- Junio_2022.csv
1.8G	data/source/INFORMACION SNBE/7- Julio_2022.csv
1.9G	data/source/INFORMACION SNBE/8- Agosto_2022.csv
1.8G	data/source/INFORMACION SNBE/9- Setiembre_2022.csv


## Cuantos registros tienen los archivos?

In [ ]:
%%bash
wc -l data/source/INFORMACION\ SNBE/*

   15137196 data/source/INFORMACION SNBE/10- Octubre_2022.csv
   15421223 data/source/INFORMACION SNBE/11- Noviembre_2022.csv
   15264875 data/source/INFORMACION SNBE/12- Diciembre_2022.csv


En fin, vamos a leerlos en un `DataFrame` de `spark` y guardarlos como parquet rápidamente.

## A ver un archivo?

In [ ]:
%%bash
head -1 data/source/INFORMACION\ SNBE/1-\ Enero_2022.csv

Al final no eran `csv` pero si un text delimited file separado por `;`.

Veo que tiene información sensible, la columna `serialtarjeta`. Es por eso que muestro solo la primera fila.

## Cargo los datos a Spark

El código en esta sección es el resultado de un proceso iterativo para asignar tipos de datos correctos a las columnas.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.getOrCreate()
snbe = spark.read.csv("data/source/INFORMACION SNBE/*.csv", sep=";", header=True)
snbe.printSchema()

snbe = (
    snbe
    .select(
        F.expr("sha2(serialtarjeta, 512)").alias("serial_tarjeta"),
        F.expr("idsam").alias("id_sam"),
        F.expr("to_timestamp(substring(fechahoraevento, 1, 23), 'yyyy/MM/dd HH:mm:ss.SSS')").alias("timestamp_evento"),
        "producto",
        F.expr("cast(montoevento as int)").alias("monto_evento"),
        F.expr("cast(consecutivoevento as int)").alias("consecutivo_evento"),
        F.expr("cast(identidad as int)").alias("identidad"),
        F.expr("cast(tipoevento as int)").alias("tipo_evento"),
        F.expr("cast(latitude as float)").alias("latitude"),
        F.expr("cast(longitude as float)").alias("longitude"),
        F.expr("idrutaestacion").alias("id_rutaestacion"),
        F.expr("tipotransporte").alias("tipo_transporte"),
    )
    .withColumn("event_year", F.expr("year(timestamp_evento)"))
    .withColumn("event_month", F.expr("month(timestamp_evento)"))
    .withColumn("event_day", F.expr("day(timestamp_evento)"))
)
snbe.printSchema()

In [ ]:
(
    snbe.write
    .partitionBy("event_year","event_month", "event_day")
    .parquet("data/raw/snbe")
)

In [ ]:
snbe = spark.read.parquet("data/raw/snbe")

In [ ]:
snbe.describe()